# 5. Influence factors: Climate on Soil Microbiome 

In [1]:
!mkdir /home/lfloerl/cloud/lfloerl/Microterroir/artifacts/16S/lavaux/climate-soil

mkdir: das Verzeichnis „/home/lfloerl/cloud/lfloerl/Microterroir/artifacts/16S/lavaux/climate-soil“ kann nicht angelegt werden: File exists


In [2]:
workdir = '/home/lfloerl/cloud/lfloerl/Microterroir/artifacts/16S/lavaux/climate-soil'
%cd $workdir

/home/lfloerl/cloud/lfloerl/Microterroir/artifacts/16S/lavaux/climate-soil


In [3]:
%env TMPDIR=/scratch/lfloerl/tmpdata

env: TMPDIR=/scratch/lfloerl/tmpdata


In [4]:
from qiime2 import Visualization
import qiime2 as q2
from qiime2 import Visualization
from qiime2 import Artifact
from qiime2 import Metadata
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from qiime2.plugins.diversity.visualizers import alpha_group_significance
from qiime2.plugins.feature_table.methods import (filter_samples, filter_seqs)

from qiime2.plugins import diversity as q2d

%matplotlib inline

# Diversity

In [7]:
#!rm -r cm1100 kmer_cm1100

In [9]:
%%bash 

# subset to make sure that all samples have climate measurements
qiime feature-table filter-samples \
      --i-table /home/lfloerl/cloud/lfloerl/Microterroir/artifacts/16S/lavaux/soil_filtered_table.qza \
      --m-metadata-file /home/lfloerl/microterroir/Microbiome/Metadata/16S_Lavaux_Climate.tsv \
      --p-where "[average_temperature] IS NOT NULL" \
      --o-filtered-table climate_filtered_table.qza

qiime feature-table filter-seqs \
        --i-data /home/lfloerl/cloud/lfloerl/Microterroir/artifacts/16S/bac-dada2-single/dada-rep-seqs-220-ee4-fa4.qza \
        --i-table climate_filtered_table.qza \
        --o-filtered-data climate_filtered_rep_seqs.qza
        
        

# Core diversity metrics
qiime diversity core-metrics \
        --i-table climate_filtered_table.qza \
        --p-sampling-depth 1100 \
        --m-metadata-file /home/lfloerl/microterroir/Microbiome/Metadata/16S_Lavaux_Climate.tsv \
        --p-n-jobs 5 \
        --output-dir cm1100/

# Kmer diversity
qiime kmerizer core-metrics \
    --i-sequences climate_filtered_rep_seqs.qza \
    --i-table climate_filtered_table.qza \
    --p-sampling-depth 1100 \
    --m-metadata-file /home/lfloerl/microterroir/Microbiome/Metadata/16S_Lavaux_Climate.tsv \
    --p-n-jobs 'auto' \
    --p-max-features 5000 \
    --output-dir kmer_cm1100/

Saved FeatureTable[Frequency] to: climate_filtered_table.qza
Saved FeatureData[Sequence] to: climate_filtered_rep_seqs.qza
Saved FeatureTable[Frequency] to: cm1100/rarefied_table.qza
Saved SampleData[AlphaDiversity] to: cm1100/observed_features_vector.qza
Saved SampleData[AlphaDiversity] to: cm1100/shannon_vector.qza
Saved SampleData[AlphaDiversity] to: cm1100/evenness_vector.qza
Saved DistanceMatrix to: cm1100/jaccard_distance_matrix.qza
Saved DistanceMatrix to: cm1100/bray_curtis_distance_matrix.qza
Saved PCoAResults to: cm1100/jaccard_pcoa_results.qza
Saved PCoAResults to: cm1100/bray_curtis_pcoa_results.qza
Saved Visualization to: cm1100/jaccard_emperor.qzv
Saved Visualization to: cm1100/bray_curtis_emperor.qzv
Saved FeatureTable[Frequency] to: kmer_cm1100/rarefied_table.qza
Saved FeatureTable[Frequency] to: kmer_cm1100/kmer_table.qza
Saved SampleData[AlphaDiversity] to: kmer_cm1100/observed_features_vector.qza
Saved SampleData[AlphaDiversity] to: kmer_cm1100/shannon_vector.qza
Sav

# Mantel test with climate euclidian distance

### 1. Median temperature & RH 

In [24]:
%%bash 

div_dir='cm1100'
kmer_dir='kmer_cm1100'
output_dir="climate_mantel"

mkdir -p $output_dir

qiime diversity mantel \
    --i-dm1 /home/lfloerl/cloud/lfloerl/Microterroir/artifacts/16S/16S_euclidian_Temp_RH.qza \
    --i-dm2 $div_dir/bray_curtis_distance_matrix.qza \
    --p-intersect-ids \
    --o-visualization $output_dir/bray_curtis_mantel.qzv
    
qiime diversity mantel \
    --i-dm1 /home/lfloerl/cloud/lfloerl/Microterroir/artifacts/16S/16S_euclidian_Temp_RH.qza \
    --i-dm2 $div_dir/jaccard_distance_matrix.qza \
    --p-intersect-ids \
    --o-visualization $output_dir/jaccard_mantel.qzv
    
qiime diversity mantel \
    --i-dm1 /home/lfloerl/cloud/lfloerl/Microterroir/artifacts/16S/16S_euclidian_Temp_RH.qza \
    --i-dm2 $kmer_dir/bray_curtis_distance_matrix.qza \
    --p-intersect-ids \
    --o-visualization $output_dir/bray_curtis_kmer_mantel.qzv
    
qiime diversity mantel \
    --i-dm1 /home/lfloerl/cloud/lfloerl/Microterroir/artifacts/16S/16S_euclidian_Temp_RH.qza \
    --i-dm2 $kmer_dir/jaccard_distance_matrix.qza \
    --p-intersect-ids \
    --o-visualization $output_dir/jaccard_kmer_mantel.qzv

Saved Visualization to: climate_mantel/bray_curtis_mantel.qzv
Saved Visualization to: climate_mantel/jaccard_mantel.qzv
Saved Visualization to: climate_mantel/bray_curtis_kmer_mantel.qzv
Saved Visualization to: climate_mantel/jaccard_kmer_mantel.qzv


In [25]:
Visualization.load('climate_mantel/bray_curtis_mantel.qzv')

<visualization: Visualization uuid: 0e78dc44-bde2-4226-85c4-760b12021b5c>

In [26]:
Visualization.load('climate_mantel/jaccard_mantel.qzv')

<visualization: Visualization uuid: 62f1a9cd-fd2f-412f-8406-afc994eed691>

In [27]:
Visualization.load('climate_mantel/bray_curtis_kmer_mantel.qzv')

<visualization: Visualization uuid: 6eeb75fe-5761-4d51-82bb-a708f622676f>

In [28]:
Visualization.load('climate_mantel/jaccard_kmer_mantel.qzv')

<visualization: Visualization uuid: 3042d58c-676f-4126-843c-98396a6affdf>

### 2. Median, min and max temperature 

In [29]:
%%bash 

div_dir='cm1100'
kmer_dir='kmer_cm1100'
output_dir="temp_mantel"

mkdir -p $output_dir

qiime diversity mantel \
    --i-dm1 /home/lfloerl/cloud/lfloerl/Microterroir/artifacts/16S/16S_euclidian_Temp.qza \
    --i-dm2 $div_dir/bray_curtis_distance_matrix.qza \
    --p-intersect-ids \
    --o-visualization $output_dir/bray_curtis_mantel.qzv
    
qiime diversity mantel \
    --i-dm1 /home/lfloerl/cloud/lfloerl/Microterroir/artifacts/16S/16S_euclidian_Temp.qza \
    --i-dm2 $div_dir/jaccard_distance_matrix.qza \
    --p-intersect-ids \
    --o-visualization $output_dir/jaccard_mantel.qzv
    
qiime diversity mantel \
    --i-dm1 /home/lfloerl/cloud/lfloerl/Microterroir/artifacts/16S/16S_euclidian_Temp.qza \
    --i-dm2 $kmer_dir/bray_curtis_distance_matrix.qza \
    --p-intersect-ids \
    --o-visualization $output_dir/bray_curtis_kmer_mantel.qzv
    
qiime diversity mantel \
    --i-dm1 /home/lfloerl/cloud/lfloerl/Microterroir/artifacts/16S/16S_euclidian_Temp.qza \
    --i-dm2 $kmer_dir/jaccard_distance_matrix.qza \
    --p-intersect-ids \
    --o-visualization $output_dir/jaccard_kmer_mantel.qzv

Saved Visualization to: temp_mantel/bray_curtis_mantel.qzv
Saved Visualization to: temp_mantel/jaccard_mantel.qzv
Saved Visualization to: temp_mantel/bray_curtis_kmer_mantel.qzv
Saved Visualization to: temp_mantel/jaccard_kmer_mantel.qzv


In [30]:
Visualization.load('temp_mantel/bray_curtis_mantel.qzv')

<visualization: Visualization uuid: 1a84f0d3-074d-44fc-a640-8368be4360bf>

In [31]:
Visualization.load('temp_mantel/jaccard_mantel.qzv')

<visualization: Visualization uuid: d469ad2b-0f5a-4d1c-8cd2-b7b1c7c15114>

In [32]:
Visualization.load('temp_mantel/bray_curtis_kmer_mantel.qzv')

<visualization: Visualization uuid: b6af0155-1f01-4378-9c4a-a6c422a406fb>

In [33]:
Visualization.load('temp_mantel/jaccard_kmer_mantel.qzv')

<visualization: Visualization uuid: dc879136-b7b9-4e3a-b2ff-e5c5a76fead2>

### 3. Median, min and max RH 

In [39]:
%%bash 

div_dir='cm1100'
kmer_dir='kmer_cm1100'
output_dir="RH_mantel"

mkdir -p $output_dir

qiime diversity mantel \
    --i-dm1 /home/lfloerl/cloud/lfloerl/Microterroir/artifacts/16S/16S_euclidian_RH.qza \
    --i-dm2 $div_dir/bray_curtis_distance_matrix.qza \
    --p-intersect-ids \
    --o-visualization $output_dir/bray_curtis_mantel.qzv
    
qiime diversity mantel \
    --i-dm1 /home/lfloerl/cloud/lfloerl/Microterroir/artifacts/16S/16S_euclidian_RH.qza \
    --i-dm2 $div_dir/jaccard_distance_matrix.qza \
    --p-intersect-ids \
    --o-visualization $output_dir/jaccard_mantel.qzv
    
qiime diversity mantel \
    --i-dm1 /home/lfloerl/cloud/lfloerl/Microterroir/artifacts/16S/16S_euclidian_RH.qza \
    --i-dm2 $kmer_dir/bray_curtis_distance_matrix.qza \
    --p-intersect-ids \
    --o-visualization $output_dir/bray_curtis_kmer_mantel.qzv
    
qiime diversity mantel \
    --i-dm1 /home/lfloerl/cloud/lfloerl/Microterroir/artifacts/16S/16S_euclidian_RH.qza \
    --i-dm2 $kmer_dir/jaccard_distance_matrix.qza \
    --p-intersect-ids \
    --o-visualization $output_dir/jaccard_kmer_mantel.qzv

Saved Visualization to: RH_mantel/bray_curtis_mantel.qzv
Saved Visualization to: RH_mantel/jaccard_mantel.qzv
Saved Visualization to: RH_mantel/bray_curtis_kmer_mantel.qzv
Saved Visualization to: RH_mantel/jaccard_kmer_mantel.qzv


In [40]:
Visualization.load('RH_mantel/bray_curtis_mantel.qzv')

<visualization: Visualization uuid: c65422f9-fd66-45b3-8379-8439fd31d0da>

In [41]:
Visualization.load('RH_mantel/jaccard_mantel.qzv')

<visualization: Visualization uuid: 220a9501-5316-4d6d-8595-8a271ae62134>

In [42]:
Visualization.load('RH_mantel/bray_curtis_kmer_mantel.qzv')

<visualization: Visualization uuid: 4745ac55-ff5d-4510-b8e2-7eb36a1ca96d>

In [43]:
Visualization.load('RH_mantel/jaccard_kmer_mantel.qzv')

<visualization: Visualization uuid: d903173f-d287-4b21-b1e6-ff5470222a1a>

# PERMANOVA

In [ ]:
md.to_dataframe()

In [44]:
formula = 'median_rh + median_temperature'

beta_metrics = ['bray_curtis', 'jaccard']

# PERMANOVA with core metrics 
div_dir = 'cm1100'
for m in beta_metrics:
    !qiime diversity adonis \
        --i-distance-matrix "{div_dir}/{m}_distance_matrix.qza" \
        --m-metadata-file /home/lfloerl/microterroir/Microbiome/Metadata/16S_Lavaux_Climate.tsv \
        --p-formula "{formula}" \
        --p-n-jobs 5 \
        --o-visualization "{div_dir}/adonis_{m}.qzv"
    

# PERMANOVA with kmer core metrics 
kmer_dir = 'kmer_cm1100'
for m in beta_metrics:
    !qiime diversity adonis \
        --i-distance-matrix "{kmer_dir}/{m}_distance_matrix.qza" \
        --m-metadata-file /home/lfloerl/microterroir/Microbiome/Metadata/16S_Lavaux_Climate.tsv \
        --p-formula "{formula}" \
        --p-n-jobs 5 \
        --o-visualization "{kmer_dir}/adonis_{m}.qzv"

Saved Visualization to: cm1100/adonis_bray_curtis.qzv
Saved Visualization to: cm1100/adonis_jaccard.qzv
Saved Visualization to: kmer_cm1100/adonis_bray_curtis.qzv
Saved Visualization to: kmer_cm1100/adonis_jaccard.qzv


In [45]:
Visualization.load('cm1100/adonis_bray_curtis.qzv')

<visualization: Visualization uuid: aeee0abf-1b0e-4281-a509-dcbbf75bccf6>

In [46]:
Visualization.load('cm1100/adonis_jaccard.qzv')

<visualization: Visualization uuid: dfb7ebab-bb12-462f-a240-82c177b3d4d2>

In [47]:
Visualization.load('kmer_cm1100/adonis_bray_curtis.qzv')

<visualization: Visualization uuid: fa9014fb-fcc2-450b-b323-eca86a6413ae>

In [48]:
Visualization.load('kmer_cm1100/adonis_jaccard.qzv')

<visualization: Visualization uuid: e29e9ea3-5ea3-4cc0-84f9-dc1c5b492f38>